<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/CNN/torch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn

In [84]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

print("기기 사용 {}".format(device))

기기 사용 cuda


In [85]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [86]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True,)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [87]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                     batch_size =batch_size,
                                     shuffle = True,
                                     drop_last= True)


In [88]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride= 1, padding= 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size= 2, stride= 2))

    self.layer2 = torch.nn.Sequential(
          torch.nn.Conv2d(32, 64, kernel_size= 3, stride= 1, padding= 1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size = 2, stride = 2))

    self.layer3 = torch.nn.Sequential(
          torch.nn.Conv2d(64, 128, kernel_size= 3, stride= 1, padding= 1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size= 2, stride = 2, padding = 1))

    self.fc1 = torch.nn.Linear(4*4*128, 625, bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    
    self.layer4 = torch.nn.Sequential(
          self.fc1,
          torch.nn.ReLU(),
          torch.nn.Dropout(p = 1 - self.keep_prob))

    self.fc2 = torch.nn.Linear(625, 10, bias= True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)


  def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = out.view(out.size(0), -1)
      out = self.layer4(out)
      out = self.fc2(out)
      return out

In [89]:
#test
net = CNN().to(device)
test_input = (torch.Tensor(100,1,28,28)).to(device)
test_out = net(test_input)

In [90]:
model = CNN().to(device)

In [91]:
#loss 함수 cuda로 실행
loss_fn = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [92]:
total_batch = len(data_loader)

In [93]:
#batch_size = 100, total_data = 60000
print(total_batch)

600


In [94]:
for epoch in range(training_epochs):
  avg_cost = 0  
  
  #미니 배치 단위로 꺼내오기
  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)

    optimizer.zero_grad()

    #예측값
    hypothesis = model(x)
    
    #Loss 값
    cost = loss_fn(hypothesis, y)
    
    #가중치에 대한 기울기 계산
    cost.backward()

    #기울기를 기준으로 가중치 조정
    optimizer.step()

    avg_cost += cost/total_batch

  print('epoch = {:>4}, cost = {:>.9}'.format(epoch+1, avg_cost))


    



epoch =    1, cost = 0.176343843
epoch =    2, cost = 0.0492793582
epoch =    3, cost = 0.0349964499
epoch =    4, cost = 0.0260172393
epoch =    5, cost = 0.0222170558
epoch =    6, cost = 0.0193982981
epoch =    7, cost = 0.0161318015
epoch =    8, cost = 0.0140052801
epoch =    9, cost = 0.0107045565
epoch =   10, cost = 0.0102375308
epoch =   11, cost = 0.00947280135
epoch =   12, cost = 0.00811980013
epoch =   13, cost = 0.00900143851
epoch =   14, cost = 0.00844216626
epoch =   15, cost = 0.00659867702


In [101]:
with torch.no_grad():
  x_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
  y_test = mnist_test.test_labels.to(device)

  prediction = model(x_test)
  correct_prediction = torch.argmax(prediction, 1) == y_test
  accuracy = correct_prediction.float().mean()
  print('Acc : {}'.format(accuracy))

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Acc : 0.9840999841690063
